<a href="https://colab.research.google.com/github/sarahaptumich/capstone/blob/shaptonstall/data_exploration.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
#Mount google drive
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
%cd /content/drive/MyDrive/Github/capstone

/content/drive/MyDrive/Github/capstone


In [ ]:
#!git pull   #check that git us up to date
#!git checkout -b shaptonstall # create a new branch

#!git push origin shaptonstall #commit changes to my branch

In [ ]:
# to commit to github
#!git status # check untracked files
#!git add {filename} # add the files
#!git config --global user.email "sarahapt@umich.edu"
# !git commit

In [114]:
# pip install factor-analyzer

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.8/42.8 kB 609.5 kB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Installing backend dependencies ... done
  Preparing metadata (pyproject.toml) ... done
  Created wheel for factor-analyzer: filename=factor_analyzer-0.5.1-py2.py3-none-any.whl size=42564 sha256=217e41919111c0c33ebac3695ab5229abd93af5a5fb77490e9646975ff74468d
  Stored in directory: /root/.cache/pip/wheels/24/59/82/6493618e30ed1cb7a013b9e1b0c9e17de80b04dfcef4ba8a4d
Successfully built factor-analyzer


Import libraries

In [119]:
import pandas as pd
import numpy as np
import re
from datetime import datetime, timedelta
from sklearn.decomposition import PCA, KernelPCA
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import StandardScaler
import matplotlib.pyplot as plt
from factor_analyzer import FactorAnalyzer
from sklearn.decomposition import FactorAnalysis
from factor_analyzer import calculate_bartlett_sphericity, calculate_kmo
from factor_analyzer import Rotator
from factor_analyzer import ConfirmatoryFactorAnalyzer

In [4]:
!ls

 Capstone.ipynb		  README.md			       Webscraping.xlsx
 data_exploration.ipynb  'trivium_full_column_names (1).csv'


In [5]:
# Load all sheets into a dictionary of DataFrames
webscraping = pd.read_excel('Webscraping.xlsx', sheet_name=None)



In [6]:
webscraping.keys()
# webscraping['Events and8']

dict_keys(['Events and8', 'Events WDSF', 'Battles Trivium', 'Battles and8 Threefold', 'Battles and8 Single Slider', 'Battles and8 Round by Round', 'Battles and8 Traditional', 'Research Questions', 'Notes', 'Notes Columns'])

Events details

In [13]:
eventsAnd8=webscraping['Events and8'][1:].copy()
new_header= eventsAnd8.iloc[0]
eventsAnd8=eventsAnd8.drop(1)
eventsAnd8.columns = new_header
eventsAnd8.rename(columns={'Name':'Event'}, inplace=True)
eventsAnd8.drop_duplicates(subset=['Event'], inplace=True)

eventsWDSF = webscraping['Events WDSF'][1:].copy()
new_header = eventsWDSF.iloc[0]
eventsWDSF = eventsWDSF.drop(1)
eventsWDSF.columns = new_header
eventsWDSF.rename(columns={'Name': 'Event'}, inplace=True)
eventsWDSF.drop_duplicates(subset=['Event'], inplace=True)

combined_events = pd.concat([eventsAnd8, eventsWDSF], ignore_index=True)


Threefold Battles

In [20]:
#threefold battles
threefold = webscraping['Battles and8 Threefold']
threefold.rename(columns={'Unnamed: 0':'Event', 'Unnamed: 1': 'Category'}, inplace=True)
threefold[['Event', 'Category']] = threefold[['Event', 'Category']].fillna(method='ffill')
threefold.head(2)

,Event,Category,breaker 1,breaker 2,winner,battle rounds,number of judges,breaker 1 round wins,breaker 2 round wins,breaker 1 vote count,...,r3j7arti,r3j7inte,r3j8over,r3j8phys,r3j8arti,r3j8inte,r3j9over,r3j9phys,r3j9arti,r3j9inte
0,Campionato Italiano di Categoria Breaking FIDS...,1vs1 BBoy 12-15 C Top 16 KnockOut,Baruzzo,Elpablix,Baruzzo,3,3,2,1,5.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,Campionato Italiano di Categoria Breaking FIDS...,1vs1 BBoy 12-15 C Top 16 KnockOut,Yoga,Faby,Faby,2,3,0,2,1.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [25]:
pd.merge(threefold, combined_events[['Date', 'Event', 'Format']], on='Event')['Format'].unique()


array([nan], dtype=object)

Trivium Battles

In [28]:
trivium=webscraping['Battles Trivium']
trivium.rename(columns={'Unnamed: 0':'Event', 'Unnamed: 1': 'Category'}, inplace=True)
trivium[['Event', 'Category']]= trivium[['Event', 'Category']].fillna(method='ffill')


In [29]:
trivium_columns=list(trivium.columns)
exclude=['judge 1 name ', 'judge 2 name', 'judge 3 name', 'judge 4 name', 'judge 5 name', 'judge 6 name ', 'judge 7 name', 'judge 8 name', 'judge 9 name'  ]
trivium_columns= [x for x in trivium_columns if x not in exclude]
trivium=trivium[trivium_columns]
trivium['BattleID']= trivium.index

<ipython-input-29-a75688204c39>:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  trivium['BattleID']= trivium.index


In [58]:
trivium.shape

(1526, 417)

In [30]:
category_sums_blue = {
    "Creativity": trivium.filter(like="crea").sum(axis=1),
    "Musicality": trivium.filter(like="musi").sum(axis=1),
    "Performance": trivium.filter(like="perf").sum(axis=1),
    "Personality": trivium.filter(like="pers").sum(axis=1),
    "Technique": trivium.filter(like="tech").sum(axis=1),
    "Variety": trivium.filter(like="vari").sum(axis=1),
    "Overall":  trivium.filter(like="over").sum(axis=1)
}

category_sums_red = {
    "Creativity": trivium.filter(like="crea").sum(axis=1)*-1,
    "Musicality": trivium.filter(like="musi").sum(axis=1)*-1,
    "Performance": trivium.filter(like="perf").sum(axis=1)*-1,
    "Personality": trivium.filter(like="pers").sum(axis=1)*-1,
    "Technique": trivium.filter(like="tech").sum(axis=1)*-1,
    "Variety": trivium.filter(like="vari").sum(axis=1)*-1,
    "Overall":  trivium.filter(like="over").sum(axis=1)*-1
}

In [59]:
scorecardBlue=pd.DataFrame({
    "Event": trivium['Event'],
    'Category': trivium['Category'],
    "BattleID": trivium["BattleID"],
    "Side": "BLUE",
    "Breaker": trivium["breaker 2 (blue)"],
        **category_sums_blue,
})

scorecardRed=pd.DataFrame({
    "Event": trivium['Event'],
    'Category': trivium['Category'],
    "BattleID": trivium["BattleID"],
    "Side": "RED",
    "Breaker": trivium["breaker 1 (red)"],
     **category_sums_red,
})

In [63]:
scorecard = pd.concat([scorecardBlue, scorecardRed], ignore_index=True)
scorecard['Score Type']="trivium"
scorecard.shape

(3052, 13)

In [ ]:
# scorecard[scorecard['Breaker']=="Ami"].mean(), scorecard[scorecard['Breaker']=="Ami"].count()

In [ ]:
# scorecard[scorecard['Breaker']==671].mean(), scorecard[scorecard['Breaker']==671].count()

In [64]:

scorecard.shape

(3052, 14)

In [66]:
scorecard=pd.merge(scorecard, eventsAnd8[['Date', 'Event']], on='Event', how='left')
# today = scorecard['Date'].max()
# total_days = (today - scorecard['Date'].min()).days + 1  # Get the total number of days dynamically
# scorecard['Weight'] = ((today - scorecard['Date']).dt.days / total_days) * 100
# scorecard['Weight'] = 100 - scorecard['Weight']  # Reverse the weights so that newer dates have higher weights

Trivium PCA

In [80]:
# triviumFloat=
triviumScore=scorecardBlue[['Creativity', 'Musicality', 'Performance', 'Personality', 'Technique', 'Variety']]

In [81]:
#imputer also get feature name for plotting later
names=triviumScore.columns
imputer = SimpleImputer(missing_values=np.nan, strategy='mean')
triviumNorm = imputer.fit_transform(triviumScore)
features=imputer.get_feature_names_out(input_features= names)


#normalize float columns
triviumNorm= StandardScaler().fit(triviumNorm).transform(triviumNorm)


In [82]:
#set PCA
pca = PCA(n_components=3, random_state=0).fit(triviumNorm)
X_pca = pca.transform(triviumNorm)
X_pca.shape, triviumNorm.shape

((1526, 3), (1526, 6))

In [83]:
features

array(['Creativity', 'Musicality', 'Performance', 'Personality',
       'Technique', 'Variety'], dtype=object)

In [166]:
loadings_abs = np.abs(pca.components_)
pcs = ['PC1', 'PC2', 'PC3']

# Create DataFrame
pca_loadings = pd.DataFrame(loadings_abs.T, columns=pcs, index=features)

# pca_loadings.T.plot.bar(figsize=(10, 20), subplots=True)

In [167]:
print( " factor Analysis accum variance: ",np.cumsum(pca.explained_variance_ratio_)[2])

 factor Analysis accum variance:  0.8822364614685656


In [168]:
pca_loadings.style.background_gradient()

,PC1,PC2,PC3
Creativity,0.362888,0.143561,0.713032
Musicality,0.403575,0.450374,0.269483
Performance,0.469467,0.239068,0.327888
Personality,0.409212,0.452774,0.295638
Technique,0.350112,0.704047,0.190013
Variety,0.441596,0.136793,0.433527


**Factor Analysis.**
Kaiser-Meyer-Olkin (KMO) Test measures the suitability of data for factor analysis. It determines the adequacy for each observed variable and for the complete model. KMO estimates the proportion of variance among all the observed variable. Lower proportion id more suitable for factor analysis. KMO values range between 0 and 1. Value of KMO less than 0.6 is considered inadequate.

In [121]:
kmo_all,kmo_model=calculate_kmo(triviumNorm)
kmo_model

0.6828153227522676

In [ ]:
triviumNorm

In [170]:
# perform factor analysis
fa = FactorAnalyzer(3, rotation=None)
fa.fit(triviumNorm)
X_transformed = fa.fit_transform(triviumNorm)

In [171]:
print( " factor Analysis accum variance: ",fa.get_factor_variance()[2][-1], fa.get_params())

 factor Analysis accum variance:  0.7885636265232234 {'bounds': (0.005, 1), 'impute': 'median', 'is_corr_matrix': False, 'method': 'minres', 'n_factors': 3, 'rotation': None, 'rotation_kwargs': {}, 'svd_method': 'randomized', 'use_smc': True}


In [172]:
faLoadings=np.abs(fa.loadings_)
fa = ['Factor1', 'Factor2', 'Factor3']

# Create DataFrame
fa_loadings = pd.DataFrame(faLoadings, columns=pcs, index=features)

# fa_loadings.T.plot.bar(figsize=(10, 20), subplots=True)


In [173]:
fa_loadings.style.background_gradient()

,PC1,PC2,PC3
Creativity,0.584749,0.144549,0.356931
Musicality,0.669645,0.352689,0.148181
Performance,0.850926,0.149023,0.278467
Personality,0.728653,0.465250,0.282656
Technique,0.682145,0.707579,0.173523
Variety,0.840725,0.059744,0.537386


In [174]:
pca_loadings.style.background_gradient()

,PC1,PC2,PC3
Creativity,0.362888,0.143561,0.713032
Musicality,0.403575,0.450374,0.269483
Performance,0.469467,0.239068,0.327888
Personality,0.409212,0.452774,0.295638
Technique,0.350112,0.704047,0.190013
Variety,0.441596,0.136793,0.433527
